# **Predicting the length of Minecraft Sessions based on the log in time and day of the week**

## Introduction

The Pacific Laboratory or Artificial Intelligence (PLAI) is a research group based at the University of British Columbia working in the Department of Computer Science. At it's core, their research focuses on artificial intellgience (1). In their most recent project, the PLAI has been collecting data from player in a Minecraft server to create an advanced AI that can respond like a real person (2). The goal is to develop learning algorithms and scaffolding for embodied (i.e., intelligent systems with a virtual or physical body with sensing, decision-making, and acting, that allows them to interact with real-world or simulated environments) AI that can be used for Minecraft NPCs and even different fields.

This group report will attempt to answer the following question with the provided datasets:

### Can we predict the length of each session based on the log in time and day of the week?

The datasets we will be working with can be summarized in tables below.

#### Table 1. Players dataset

| Variable Name     | Type          | Description                                      |
|-------------------|---------------|--------------------------------------------------|
| experience        | Categorical   | Self-reported skill level of player              |
| subscribe         | Categorical   | Whether or not the player is subscribed to the newsletter |
| hashedEmail       | Categorical   | Unique player identification                     |
| played_hours      | Quantitative  | Total hours of playtime                          |
| name              | Categorical   | Chosen name of player                            |
| gender            | Categorical   | Chosen gender of player                          |
| age               | Categorical   | Reported age of player                           |
| individualId      | N/A           | No information                                   |
| organizationName  | N/A           | No information                                   |


#### Table 2. Sessions dataset

| Variable Name        | Type        | Description                                                                 |
|----------------------|-------------|-----------------------------------------------------------------------------|
| hashedEmail          | Categorical | Unique player identification                                                |
| start_time           | Temporal    | Timestamp of when player logs in (day/month/year/hour)                     |
| end_time             | Temporal    | Timestamp of when player logs off (day/month/year/hour)                    |
| original_start_time  | Temporal    | Exact second timestamp the player logged on since server start             |
| original_end_time    | Temporal    | Exact second timestamp the player logged off since server start            |


## Methods and Results

The important data from Table 2 is start_time and end_time, all other columns can be dropped.

In [1]:
import pandas as pd
import altair as alt

# Loading the data from the internet

url = 'https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB'
sessions = pd.read_csv(url)

# Dropping unnecessary data

sessions_times = sessions.drop(['hashedEmail', 'original_start_time', 'original_end_time'], axis = 1)

The following code cleaned up the data in order to make it more accessible to perform our data analysis:

In [2]:
# Converting to datetime to handle data properly

sessions_times['start_time'] = pd.to_datetime(sessions_times['start_time'], dayfirst = True)
sessions_times['end_time'] = pd.to_datetime(sessions_times['end_time'], dayfirst = True)

# Creating a new column for day of the week with Monday as 0 and Sunday as 6

sessions_days = sessions_times.assign(day_of_week = sessions_times['start_time'].dt.dayofweek)

# Calculating the duration of each login in minutes

sessions_days['duration_minutes'] = (sessions_days['end_time'] - sessions_days['start_time']) / pd.Timedelta(minutes=1)

# Calculating the average duration grouped by the day of the week

avg_duration = sessions_days.groupby('day_of_week')['duration_minutes'].mean().reset_index()

# Converting the start_time into measures of hours with minutes as decimals

sessions_days['start_time'] = (
    pd.to_datetime(sessions['start_time'], dayfirst=True).dt.hour +
    pd.to_datetime(sessions['start_time'], dayfirst=True).dt.minute / 60
)

With our data wrangled, we can now make relevant visualizations to help explore our data

#### Figure 1: Average duration of sessions for days of the week

In [3]:
duration_plot = alt.Chart(avg_duration).mark_bar().encode(
    x = alt.X('duration_minutes:Q').title('Average duration of each session (minutes)'),
    y = alt.Y('day_of_week:N').title('Day of the week'),
    color = alt.Color('day_of_week:N', legend = alt.Legend(title = 'Day of the week (Monday = 0)'),) #scale = alt.Scale(scheme = 'dark2'))
).properties(
    width = 345,
    height = 250,
    title = 'Average duration of sessions every day of the week'
)
duration_plot

alt.Chart(...)

This figure demonstrates that on Saturday and Sunday (days 5 and 6 indexed respectively), the sessions of players tend to be longer

#### Figure 2: Duration of sessions relative to login time

In [4]:
start_duration_all = alt.Chart(sessions_days).mark_point(opacity = 0.35).encode(
    x=alt.X('start_time:Q',
        title='Hour of the Day',
        axis=alt.Axis(values=list(range(0, 24)), tickMinStep=1)),
    y=alt.Y('duration_minutes').title('Duration of session (minutes)')
).properties(
    width = 500,
    title = 'Duration of session relative to login time'
)
start_duration_all

alt.Chart(...)

This plot illustrates the relationship between login time on the x-axis and session duration on the y-axis for all recorded player sessions. There is no strong predictive pattern with the points widely scattererd. There appears to be a higher volume of players at night with very little players during midday

#### Figure 3: Duration of sessions relative to login time faceted by days of the week

In [5]:
start_duration_days = start_duration_all.facet(
    column = alt.Column('day_of_week:N', title = 'Day of the week (Monday = 0)')
).properties(
    title = 'Duration of session in relative to login time faceted by day of the week'
)
start_duration_days

alt.FacetChart(...)

This visualization provides session duration for each day of the week (Monday-Sunday) separately to see what hour of the day is most common. Longer sessions tend to happen on Saturday and Sunday, with more than 150 minutes. This graph provides more insight (what hour of the day) than previous figures

### Creating KNN Regression Models

Our research question asks if we can predict the length of each session based on the login time and day of the week. This question naturally leads us to want to create a regression model to help predict a numeric outcome from a set of predictor variables. We chose to use a K-Nearest Neighbors regression due to the nature of the model. KNN requires minimal assumptions about the data such as not assuming linear relationships between the data. As we observe the session duration not following a direct linear pattern, KNN allows the model to adapt flexibly to this data. Furthermore, using a KNN model allows us to make clear interpretations on the data. For example, as it is based on nearest neighbors, we could potentially make interpretations such as "players who log in at similar times of the day and on similar days may have similar session lengths".

In [6]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import set_config
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

### KNN-Regression (All-Days)

Here we create a KNN-Regression for the data, using all days in the model

In [55]:
np.random.seed(10)

We set a random seed to ensure reproducibility. This ensures that the results of the train-test split and KNN model remain the same each time the code in run

In [7]:
# load data
url = 'https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB'
sessions = pd.read_csv(url)
sessions_times = sessions.drop(['hashedEmail', 'original_start_time', 'original_end_time'], axis = 1)
sessions_times = sessions_times.dropna()
# datetime conversion
sessions_times['start_dt'] = pd.to_datetime(sessions_times['start_time'], dayfirst = True)
sessions_times['end_dt'] = pd.to_datetime(sessions_times['end_time'], dayfirst = True)

The dataset is loaded from an online source file. Columns that are not needed for the predictions are removed. We then drop all rows with missing values to avoid complications during modeling. We then convert the time columns into datetime format so that we can easily extract numerical features such as hour of day and day of the week.

In [8]:
# calculating duration of sessions in minutes
sessions_times['s_duration'] = (sessions_times['end_dt'] - sessions_times['start_dt']).dt.total_seconds() / 60
# getting weekdays and hour data
sessions_times['s_weekday'] = sessions_times['start_dt'].dt.day_name()
sessions_times['s_weekday_num'] = sessions_times['start_dt'].dt.weekday
sessions_times['s_hour'] = sessions_times['start_dt'].dt.hour

We then calculate how long each session lasts and convert the result into minutes. This becomes our response variable. From the login times, we use 2 predictor variables, the day of the week, and the hour of the login. These are the features we use to predict the session duration.

In [9]:
# x and y
X = sessions_times[['s_weekday_num', 's_hour']]
y = sessions_times['s_duration']
# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3)

We then assign the predictors to X and the target variable to y to use for regression. The dataset is then split into training (70%) and testing (30%) subsets. This split was chosen to prevent overfitting and allows us to test the performance of the model on unseen data.

In [10]:
# preprocessor and knn
preprocessor = make_column_transformer(
    (StandardScaler(), ['s_weekday_num', 's_hour']))
pipeline = make_pipeline(preprocessor, KNeighborsRegressor())

We then create a pipeline to standardize the predictors before fitting it on the KNN model. This is essential because the KNN model is distance-based. This means predictors need to be on the same scale.

In [11]:
# 5-fold gridsearch
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 25, 2),
}
gridsearch = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid, 
    cv = 5, 
    scoring = 'neg_root_mean_squared_error'
)
# fit gridsearch
gridsearch.fit(X_train, y_train)

/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['s_weekday_num',
                                                                          's_hour'])])),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': range(1, 25, 2)},
             scoring='neg_root_mean_squared_error')

We then use a 5-fold cross-validation split to choose the best value of k (number of neighbors). We evaluate odd values from 1 to 23. The metric we use to score the model is RMSE (root mean squared error) which helps us measure prediction accuracy. The grid search trains multiple KNN models with different values of k and then selects the one with the lowest average RMSE value.

In [12]:
results = pd.DataFrame(gridsearch.cv_results_)
results['sem_test_score'] = results['std_test_score'] / 5**(1/2)
results = (
    results[[
        "param_kneighborsregressor__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns = {"param_kneighborsregressor__n_neighbors": "n_neighbors"})
)
results['mean_test_score'] = -results['mean_test_score']
gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 19}

We then convert all of the grid search results into a clearn table and compute the standard error of the mean RMSE. We reduce the table to only the key information. Note: we convert the RMSE value to its negative counterpart as GridSearchCV uses a negative RMSE to score values. We then retrieve the value of k that achieved the lowest cross-validation RMSE. For this model it was k=21.

In [13]:
sessions_times['predicted'] = gridsearch.predict(
    sessions_times[['s_weekday_num', 's_hour']]
)

RMSPE = mean_squared_error(
    y_true = sessions_times['s_duration'],
    y_pred = sessions_times['predicted']
)**(1/2)
RMSPE

np.float64(53.76576428382322)

After selecting the best k value, we used the KNN regression model to predict session durations for every observation in the data set. Then we compute the RMSPE to calculate how far the model's predictions are from the true session durations on average. Our RMSPE value is 53.83 which means that on average, the KNN regression model's predictions differ from the actual session durations by about 54 minutes. This is a large error relative to the scale of session lengths in the dataset. Most sessions are well under 100 minutes, and many are below 50 minutes. Therefore, a prediction error of more than 50 minutes is quite large.

### KNN Regresion (Weekends)

The same methodology was conducted but limiting the data to only weekends. We chose to conduct a "weekend-only" analysis as weekends could differ from the overall dataset as people tend to have more free-time on weekends. The same methodology of creating a KNN regression model, finding the best k value using a 5-fold cross-validation split, then calculating the RMPSE of the model using the best k value was conducted.

In [14]:
np.random.seed(10)

wkd_sessions = sessions_times[
    sessions_times['s_weekday_num'].isin([5, 6])].dropna()  # sat = 5, sun = 6


X_wkd = wkd_sessions[['s_weekday_num', 's_hour']]
y_wkd= wkd_sessions['s_duration']

wkd_X_train, wkd_X_test, wkd_y_train, wkd_y_test = train_test_split(
    X_wkd, y_wkd, test_size = 0.3
)

wkd_preprocessor = make_column_transformer(
    (StandardScaler(), ['s_weekday_num', 's_hour']))
wkd_pipeline = make_pipeline(wkd_preprocessor, KNeighborsRegressor())

param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 25, 2),
}
wkd_gridsearch = GridSearchCV(
    estimator = wkd_pipeline,
    param_grid = param_grid, 
    cv = 5, 
    scoring = 'neg_root_mean_squared_error',
    #n_jobs = -1
)

# fit gridsearch
wkd_gridsearch.fit(wkd_X_train, wkd_y_train)

wkd_results = pd.DataFrame(wkd_gridsearch.cv_results_)
wkd_results['sem_test_score'] = wkd_results['std_test_score'] / 5**(1/2)
wkd_results = (
    wkd_results[[
        "param_kneighborsregressor__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns = {"param_kneighborsregressor__n_neighbors": "n_neighbors"})
)

wkd_results['mean_test_score'] = -wkd_results['mean_test_score']

wkd_gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 21}

In [15]:
wkd_sessions['predicted'] = wkd_gridsearch.predict(
    wkd_sessions[['s_weekday_num', 's_hour']]
)

RMSPE = mean_squared_error(
    y_true = wkd_sessions['s_duration'],
    y_pred = wkd_sessions['predicted']
)**(1/2)
RMSPE


np.float64(55.95489992996134)

Our model found a RMSPE value of 55.95 which means the weekend regression model's prediction differ from the actual session durations by about 56 mintues on average. This level of error is extremely high relative to the typical range of session lengths in the dataset. This tells us that weekend session behavior appears to be even more variable and unpredictable than the overall dataset. There does not appear to be any consistent pattern linking login hour or day to how long players stay logged in.

### KNN Regression (Weekdays)

The same methodology was conducted but limiting the data to only weekdays. We chose to conduct a "weekday-only" analysis as weekdays could differ from the overall dataset as people tend to have less free-time on weekends. The same methodology of creating a KNN regression model, finding the best k value using a 5-fold cross-validation split, then calculating the RMPSE of the model using the best k value was conducted.

In [18]:
np.random.seed(10)

# filter for weekdays
wkdy_sessions = sessions_times[
    sessions_times['s_weekday_num'].isin([0, 1, 2, 3, 4])
].dropna()

X_wkdy = wkdy_sessions[['s_weekday_num', 's_hour']]
y_wkdy = wkdy_sessions['s_duration']

wkdy_X_train, wkdy_X_test, wkdy_y_train, wkdy_y_test = train_test_split(
    X_wkdy, y_wkdy, test_size = 0.3
)

wkdy_preprocessor = make_column_transformer(
    (StandardScaler(), ['s_weekday_num', 's_hour'])
)
wkdy_pipeline = make_pipeline(wkdy_preprocessor, KNeighborsRegressor())

wkdy_param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 25, 2),
}

wkdy_gridsearch = GridSearchCV(
    estimator = wkdy_pipeline,
    param_grid = wkdy_param_grid, 
    cv = 5, 
    scoring = 'neg_root_mean_squared_error',
    # n_jobs = -1
)

# fit gridsearch
wkdy_gridsearch.fit(wkdy_X_train, wkdy_y_train)

wkdy_results = pd.DataFrame(wkdy_gridsearch.cv_results_)
wkdy_results['sem_test_score'] = wkdy_results['std_test_score'] / 5**(1/2)
wkdy_results = (
    wkdy_results[[
        "param_kneighborsregressor__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns = {"param_kneighborsregressor__n_neighbors": "n_neighbors"})
)

wkdy_results['mean_test_score'] = -wkdy_results['mean_test_score']

wkdy_gridsearch.best_params_


{'kneighborsregressor__n_neighbors': 23}

In [17]:
wkdy_sessions['predicted'] = wkdy_gridsearch.predict(
    wkdy_sessions[['s_weekday_num', 's_hour']]
)

wkdy_RMSPE = mean_squared_error(
    y_true = wkdy_sessions['s_duration'],
    y_pred = wkdy_sessions['predicted']
)**(1/2)
wkdy_RMSPE

np.float64(53.02940568069593)

Similar to the past 2 models, our model found a RMSPE value of 53.03 which means the weekday regression model's prediction differ from the actual session durations by about 53 mintues on average. This level of error is extremely high relative to the typical range of session lengths in the dataset. In the same way as both models above, the weekday model cannot give us any concrete insights into weekday session behavior, showing no consistent pattern linking the login hour or day to how long players stay logged in.

### Visualzation of the data analysis

In [31]:
best_k_week = gridsearch.best_params_['kneighborsregressor__n_neighbors']
best_week_model = gridsearch.best_estimator_

sessions_days = sessions_times.assign(
    day_of_week = sessions_times['start_dt'].dt.dayofweek
)

sessions_days['duration_minutes'] = (sessions_days['end_dt'] - sessions_days['start_dt']) / pd.Timedelta(minutes=1)

sessions_days['start_time'] = (
    pd.to_datetime(sessions_days['start_dt'], dayfirst=True).dt.hour +
    pd.to_datetime(sessions_days['start_dt'], dayfirst=True).dt.minute / 60
)

week_prediction_grid = pd.DataFrame({
    's_hour': np.arange(
        sessions_days['s_hour'].min(), 
        sessions_days['s_hour'].max() + 1
    )
})

week_prediction_grid['s_weekday_num'] = 1 # will change what pred line for the day (tuesday used)

week_prediction_grid["predicted"] = best_week_model.predict(
    week_prediction_grid[["s_weekday_num", "s_hour"]]
)

start_duration_all = alt.Chart(sessions_days).mark_point(opacity = 0.35).encode(
    x = alt.X('start_time:Q',
        title = 'Hour of the Day',
        axis = alt.Axis(values=list(range(0, 24)), tickMinStep=1)),
    y=alt.Y('duration_minutes').title('Duration of session (minutes)'),
    color = alt.Color('s_weekday:N', title='Day of Week'),
).properties(
    width = 500,
    title = 'Duration of session relative to login time'
)
start_duration_all

start_duration_preds_plot = start_duration_all + alt.Chart(
    week_prediction_grid, 
).mark_line(
    color = 'black'
).encode(
    x = 's_hour:Q',
    y = 'predicted:Q'
).properties(
    title = f"K {best_k_week}"
)
start_duration_preds_plot

alt.LayerChart(...)

## Discussion

#### Summary of Findings

Across all 3 constructued KNN-Models (alldays, weekends, weekdays), we found that the login time (hour of the day) and the day of the week are not useful predictors of session duration. The KNN regression models produced RMSPEs of 53.83 minutes, 55.95 minutes, and 53.03 minutes for all days, weekends, and weekdays respectively. These errors are extremely large relative to the typical session lengths, meaning that the model's predictions are on average, off by nearly an hour. The scatterplots showed high variability in session durations across all hours and days. Overall, the results consistently demonstrate that there is no strong relationship between login time and how long a player stays in the session. 

#### Were These Results Expected?

To some extent, these results were expected. Login time and the day of the week are very high-level variables. They struggle to capture a player's motivations, experience level, or gameplay activity, which are all variables that likely have a greater influence on session length. For example, if a player's only motivation of logging on is to check on something inside of the game, the session time will likely be very short. Similarly, if a player logs on with the intention of making large progressions, their session length may be much longer. This is an example of a variable that could not be captured and was not accounted for in our mode. However, we expected at least some pattern, such as longer sessions during weekends or evenings. The results show that even these intuitive patterns are weak. Session duration remains highly variable regardless of when players log in. Thus, while the direction of these findings we partially expected, the degree of the observed unpredictability was more extreme than initially anticipated.

#### What is the Impact of These Findings?

These findings suggest several implications despite not producing an accuate predictive model. Our lack of ability to predict session length shows that time-based analyses are insufficent. This is important because for example, if a game developer wants to predict player engagement based solely on time-based activity, our results show that such predictions would not be reliable. Furthermore, these findings lead us to understand the importance of more complex behavioral data required. As previously alluded to, a player's motivation would likely be much more predictive than a time-based predictor. 

#### What Future Questions Could This Lead To?

The observed lack of predictive power form our time-based features raise new research questions that could be beneficial. As our model was unable to make accurate predictions on session length, the next reasonable question would be what variables *do* predict session duration? This leads us to be curious on what factors could be more predictive. Furthermore, this leads to questioning if individual players may show predictable patterns even if the overall population does not. An individual may log in every saturday at 5pm and play until their sports practice at 7pm for example. For an individual, time-based analysis could be more predictive while patterns like these are not seen in the overall population dataset as the variability of individuals' schedules is immense. A final question that this analysis could lead to would be asking if a different regression model could capture our question more effectively. Although we believe a linear regression model would not work as well due to the non-linear character of our data, we wonder if further predictive models exist that could potentially lead to better predictions than our choice of a KNN-regression model.

#### Overall Takeaway

Overall, our analysis strongly suggests that session duration is not driven by login time or by the day of the week, and that more meaningful variables are likely necessary to achieve accurate predictions on session length. These findings highlight the complexity of player activity and lead to further questions and deeper investigation.

## Citations

- PLAI: People + AI Research at UBC. (n.d.). Retrieved from https://plai.cs.ubc.ca
- Plaicraft. (n.d.). Retrieved from https://plaicraft.ai/